In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision


import numpy as np
import matplotlib.pyplot as plt

from data.EuroNotes import EuroNotes
from models.ResNet18 import pretrained_res18

In [2]:
# # # # # # # # # # # # # # # #
# Load Classifier Models
# # # # # # # # # # # # # # # #


print('Loading base classifier model and robust model')
model_base = pretrained_res18()
model_robust = pretrained_res18()
print('Loaded')

Loading base classifier model and robust model
Loaded


In [3]:
print(model_base)

ResNet(
  (conv1): Conv2d (3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNo

In [4]:
means = np.array([ 0.14588552,  0.26887908,  0.14538361])
stds = np.array([ 0.20122388,  0.2800698 ,  0.20029236])
transformations = transforms.Compose([transforms.ToTensor(), transforms.Normalize(means, stds)])

train_set = EuroNotes('../data-augmentation/banknotes_augmented_small/train', transform=transformations, resize=False)
val_set = EuroNotes('../data-augmentation/banknotes_augmented_small/val', transform=transformations, resize=False)

train_loader = DataLoader(train_set, batch_size=25, shuffle=True, num_workers=16)
val_loader = DataLoader(val_set, batch_size=25, shuffle=True, num_workers=16)

In [5]:
import matplotlib.gridspec as gridspec

gs = gridspec.GridSpec(2, 2)
plt.figure(figsize=(20,10))

for i_batch, data in enumerate(train_loader):
    if i_batch > 3:
        break
    images = Variable(data['image'])
    print(images.shape)
    print(type(images))
    preds = model_base(images)
    print(preds[:3, :])
#     plt.title(i_batch)
#     im = images[1, :, :, :].data.numpy() * stds.reshape((3, 1, 1))
    
#     im = im + means.reshape((3, 1, 1))
#     im = im.astype(float)
#     im = im.astype(float)
#     im -= np.min(im)
#     im /= np.max(im)
#     im = np.swapaxes(np.swapaxes(im, 0, 2).reshape((256, 256, 3)), 0, 1)
#     ax = plt.subplot(gs[int(i_batch/2), i_batch % 2])
#     plt.imshow(im)
# plt.show()
    

torch.Size([25, 3, 256, 256])
<class 'torch.autograd.variable.Variable'>


RuntimeError: size mismatch, m1: [25 x 2048], m2: [512 x 4] at /pytorch/torch/lib/TH/generic/THTensorMath.c:1416

In [ ]:
from attackers.BlackBoxAttacker import GANAttack

In [6]:
ganatt = GANAttack((3, 256, 256), 0.01)

Loading pre-trained GANAttack


In [7]:
images.shape

torch.Size([25, 3, 256, 256])

In [8]:
preds = model_base(images)
print(preds.shape)

torch.Size([25, 4])


In [9]:
np.random.uniform(-1, 0.1, (3, 2))

array([[-0.23771642, -0.09131742],
       [-0.29995057, -0.20065749],
       [-0.25190148, -0.07366671]])